# Get Audio Features from Spotify's Web API

In [6]:
# Useful imports
import spotipy # Package to retrieve the data from the Spotify Web API
import spotipy.oauth2
import pandas as pd
import json

## 1. Set-up the Spotify Web API Credentials

In [7]:
# Save my credentials into a variable <client_id, client_secret>
credentials = spotipy.oauth2.SpotifyClientCredentials("fd112ec05d9b4dbebb22542f9dd2ba6c", "aaeec71cda0a4af389e45ec29260ec18")

In [8]:
# Save the authenticator
spotify = spotipy.Spotify(client_credentials_manager=credentials)

## 2. Pre-processing the soundtacks data

In [9]:
# Import the soundtracks dataset
soundtracks = pd.read_csv("/Users/joaomanso/Desktop/Master Thesis/Data & ML/Data/Spotify Million Playlist/soundtrack_data.csv")

# Remove 'spotify:track:' from each track_id to retrieve the information
tracks_id = soundtracks['track_uri'].str.replace("spotify:track:", "")

# Set up the number of observations that will be passed
tracks_id = pd.DataFrame(tracks_id)


# Test with fewer observations
#tracks_id_100 = tracks_id[:100]
#tracks_id_test = ",".join(tracks_id_100)
#tracks = ",".join(tracks_id)


In [10]:
# Testing with 100 features
#audio_features_100 = spotify.audio_features(tracks_id_test)
#audio_features_100 = pd.DataFrame(audio_features_100)
#audio_features_100

## 3. Accessing Audio Features

In [11]:
# Create a nested loop to iteratively retrieve 100 observations at each time
batchsize = 100
audio_features = []

for i in range(0,len(tracks_id['track_uri']),batchsize):
    batch = tracks_id['track_uri'][i:i+batchsize]
    feature_results = spotify.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            audio_features.append(t)

In [59]:
# Transform to a DataFrame
audio_features_df = pd.DataFrame(audio_features)

# Drop useless attributes
audio_features_df = audio_features_df.drop(columns=['analysis_url', 'duration_ms', 'id', 'track_href', 'type'])

# Rename the attribute uri to track_uri
audio_features_df.columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
                            'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'track_uri', 'valence']

# Save as a CSV file
audio_features_df.to_csv("/Users/joaomanso/Desktop/Master Thesis/Data & ML/Data/Spotify Million Playlist/audio_features.csv", index=False, sep=";")

## 4. Merging the two datasets

In [63]:
# Merging the 2 datasets <sountrack, audio_features>
soundtrack_audio_features = pd.merge(audio_features_df, soundtracks, on='track_uri') 

In [103]:
# Removing all the duplicate values
soundtrack_audio_features = soundtrack_audio_features.drop_duplicates()

# Create a new and cleaned dataframe
soundtrack_audio_features = pd.DataFrame(soundtrack_audio_features)

# Save the final file to CSV
soundtrack_audio_features.to_csv('/Users/joaomanso/Desktop/Master Thesis/Data & ML/Data/Spotify Million Playlist/soundtrack_audio_features.csv')

In [102]:
unique_pid = len(soundtrack_audio_features['pid'].unique().tolist())
unique_tracks = len(soundtrack_audio_features['track_uri'].unique().tolist())


print('unique playlists : ' ,unique_pid)
print('unique tracks : ' ,unique_tracks)


('unique playlists : ', 4000)
('unique tracks : ', 93139)
